In [0]:
# Libraries & Data Imports

In [0]:
!curl 'https://cdn.skillenza.com/files/8a6674bb-25af-475c-84b5-4f5d103593ec/training_1.csv' -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:69.0) Gecko/20100101 Firefox/69.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' --compressed -H 'Connection: keep-alive' -H 'Referer: https://skillenza.com/challenge/datasemantics-hiringchallenge-datascientist-sep/checkpoint/submit/1933' -H 'Cookie: _ga=GA1.2.1778547320.1565265892; _hjid=8c788acc-d536-49ca-ad54-9b1baf038076; G_ENABLED_IDPS=google; _gid=GA1.2.210378034.1569058731' -H 'Upgrade-Insecure-Requests: 1' -H 'TE: Trailers' -o train.csv
!curl 'https://cdn.skillenza.com/files/a3344af0-f63d-43c4-a2e6-007175a6ebb2/test.csv' -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:69.0) Gecko/20100101 Firefox/69.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' --compressed -H 'Connection: keep-alive' -H 'Referer: https://skillenza.com/challenge/datasemantics-hiringchallenge-datascientist-sep/checkpoint/submit/1933' -H 'Cookie: _ga=GA1.2.1778547320.1565265892; _hjid=8c788acc-d536-49ca-ad54-9b1baf038076; G_ENABLED_IDPS=google; _gid=GA1.2.210378034.1569058731' -H 'Upgrade-Insecure-Requests: 1' -H 'TE: Trailers' -o test.csv

In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [0]:
import warnings
warnings.filterwarnings("ignore") 

from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTEENN,SMOTETomek

In [0]:
df_raw = pd.read_csv("train.csv", low_memory=False)
test_data = pd.read_csv("test.csv", low_memory=False)

In [0]:
my_dict = {}
for each in df_raw['text']:
  temp_list = each.split()
  for word in temp_list:
    if word in my_dict:
      my_dict[word] += 1
    else:
      my_dict.update({word: 1})

In [9]:
from operator import itemgetter  

i = 0
for key, value in sorted(my_dict.items(), key = itemgetter(1), reverse = True):
    print(key, value)
    i += 1
    if i == 20:
      break

5773 3131
237 1267
2734 1255
230 1251
6084 1115
3078 1069
7526 949
8542 797
5922 795
4017 758
2698 615
5071 592
6300 576
224 573
5045 547
753 541
6700 523
8406 455
4211 419
1994 400


In [110]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1',
    ngram_range=(1, 2), stop_words=['5773', '237', '2734', '230', '6084', '3078'])
features = tfidf.fit_transform(df_raw.text).toarray()
labels = df_raw.category
features.shape

(3464, 2395)

In [0]:
#Splitting data
X_train, X_test, y_train, y_test = train_test_split(
    df_raw['text'], df_raw['category'], test_size=0.3)

# Transforming Text data
count_vect = CountVectorizer(stop_words=['5773'])
X_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)


In [112]:
# Without Resampling
clf = LinearSVC().fit(X_tfidf, y_train)
y_pred = clf.predict(count_vect.transform(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      1001
           1       0.52      0.97      0.68        39

    accuracy                           0.97      1040
   macro avg       0.76      0.97      0.83      1040
weighted avg       0.98      0.97      0.97      1040



In [20]:
df_raw.groupby('category').count()

,id,text
category,,
0,3348,3348
1,116,116


## Data is Imbalanced:
Applying Some Resampling techniques

In [106]:
# SMOTE oversampling data
os=SMOTE(ratio=0.5,random_state=1,k_neighbors=5)
X_train_res, y_train_res=os.fit_sample(X_tfidf, y_train)
clf = LinearSVC().fit(X_train_res, y_train_res)
y_pred = clf.predict(count_vect.transform(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      1014
           1       0.46      1.00      0.63        26

    accuracy                           0.97      1040
   macro avg       0.73      0.99      0.81      1040
weighted avg       0.99      0.97      0.98      1040



In [33]:
# SMOTE (OverSampling) + Tomek Link Removal (UnderSampling)
os_us=SMOTETomek(ratio=0.5,random_state=1)
X_train_res, y_train_res=os_us.fit_sample(X_tfidf, y_train)
clf = LinearSVC().fit(X_train_res, y_train_res)
y_pred = clf.predict(count_vect.transform(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      1002
           1       0.52      0.97      0.68        38

    accuracy                           0.97      1040
   macro avg       0.76      0.97      0.83      1040
weighted avg       0.98      0.97      0.97      1040



In [34]:
# SMOTE (OverSampling) + Edited Nearest Neighbor (UnderSampling)

os_us=SMOTEENN(ratio=0.5,random_state=1)
X_train_res, y_train_res=os_us.fit_sample(X_tfidf, y_train)
clf = LinearSVC().fit(X_train_res, y_train_res)
y_pred = clf.predict(count_vect.transform(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.88      0.93      1002
           1       0.23      0.97      0.37        38

    accuracy                           0.88      1040
   macro avg       0.61      0.92      0.65      1040
weighted avg       0.97      0.88      0.91      1040



# Model Selection

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_features='sqrt', min_samples_leaf=4, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  precisions = cross_val_score(model, features, labels, scoring='precision', cv=CV)
  recalls = cross_val_score(model, features, labels, scoring='recall', cv=CV)
  f1s = cross_val_score(model, features, labels, scoring='f1', cv=CV)
  for fold_idx, score in enumerate(zip(accuracies, precisions, recalls, f1s)):
    entries.append((model_name, fold_idx, score[0], score[1], score[2], score[3]))
cv_df = pd.DataFrame(entries,
                     columns=['model_name', 'fold_idx', 'accuracy', 'precision', 'recall', 'f1'])

In [0]:
cv_df

,model_name,fold_idx,accuracy,precision,recall,f1
0,RandomForestClassifier,0,0.968300,1.000000,0.083333,0.153846
1,RandomForestClassifier,1,0.971140,1.000000,0.130435,0.230769
2,RandomForestClassifier,2,0.974026,1.000000,0.217391,0.357143
3,RandomForestClassifier,3,0.973988,1.000000,0.217391,0.357143
4,RandomForestClassifier,4,0.971098,1.000000,0.130435,0.230769
5,LinearSVC,0,0.989914,1.000000,0.708333,0.829268
6,LinearSVC,1,0.987013,1.000000,0.608696,0.756757
7,LinearSVC,2,0.997114,1.000000,0.913043,0.954545
8,LinearSVC,3,0.986994,0.888889,0.695652,0.780488
9,LinearSVC,4,0.992775,1.000000,0.782609,0.878049


# Working On Random Forest

In [0]:
rf_clf = RandomForestClassifier(
    n_estimators=50, max_features='sqrt', min_samples_leaf=10)
rf_clf.fit(X_train_res, y_train_res)
y_pred = rf_clf.predict(count_vect.transform(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1008
           1       0.74      0.81      0.78        32

    accuracy                           0.99      1040
   macro avg       0.87      0.90      0.88      1040
weighted avg       0.99      0.99      0.99      1040



# Trying Neural Network

In [87]:
from sklearn.neural_network import MLPClassifier

nn_clf = MLPClassifier(hidden_layer_sizes=10, activation='relu')
nn_clf.fit(X_tfidf, y_train)
y_pred = nn_clf.predict(count_vect.transform(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1014
           1       0.95      0.73      0.83        26

    accuracy                           0.99      1040
   macro avg       0.97      0.86      0.91      1040
weighted avg       0.99      0.99      0.99      1040



In [88]:
# SMOTE (OverSampling) + Edited Nearest Neighbor (UnderSampling)

os_us=SMOTEENN(ratio=0.5,random_state=1)
X_train_res, y_train_res=os_us.fit_sample(X_tfidf, y_train)
nn_clf = MLPClassifier(hidden_layer_sizes=10, activation='relu')
nn_clf.fit(X_train_res, y_train_res)
y_pred = nn_clf.predict(count_vect.transform(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1014
           1       0.56      0.92      0.70        26

    accuracy                           0.98      1040
   macro avg       0.78      0.95      0.84      1040
weighted avg       0.99      0.98      0.98      1040



In [99]:
# SMOTE (OverSampling) + Tomek Link(UnderSampling)

os_us=SMOTETomek(ratio=0.5,random_state=1)
X_train_res, y_train_res=os_us.fit_sample(X_tfidf, y_train)
nn_clf = MLPClassifier(hidden_layer_sizes=50, activation='relu')
nn_clf.fit(X_train_res, y_train_res)
y_pred = nn_clf.predict(count_vect.transform(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1014
           1       0.88      0.88      0.88        26

    accuracy                           0.99      1040
   macro avg       0.94      0.94      0.94      1040
weighted avg       0.99      0.99      0.99      1040



In [105]:
# SMOTE oversampling data
os=SMOTE(ratio=0.5,random_state=1,k_neighbors=5)
X_train_res, y_train_res=os.fit_sample(X_tfidf, y_train)
nn_clf = MLPClassifier(hidden_layer_sizes=50, activation='tanh')
nn_clf.fit(X_tfidf, y_train)
y_pred = nn_clf.predict(count_vect.transform(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1014
           1       0.96      0.88      0.92        26

    accuracy                           1.00      1040
   macro avg       0.98      0.94      0.96      1040
weighted avg       1.00      1.00      1.00      1040



# Test Result Output

In [0]:
out = test_data.copy()
out['category'] = clf.predict(count_vect.transform(out['text']))
out[['id', 'category']].to_csv('test_result.csv', index=False)

In [0]:
type(X_tfidf[0])

scipy.sparse.csr.csr_matrix